In [3]:
#recognize GPU
import tensorflow as tf
tf.config.list_physical_devices('GPU')

#config = tf.ConfigProto()
#config.gpu_options.allow_growth = True
#sess = tf.Session(config=config)

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [4]:
import tensorflow as tf

In [5]:
#Download dataset (do only once)
!ls -lha kaggle.json
%pip install -q kaggle
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d datasnaek/chess

/usr/bin/bash: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)
-rw-rw-r-- 1 lucas lucas 65 Nov 29 14:25 kaggle.json
/usr/bin/bash: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)
^C
ERROR: Operation cancelled by user
Note: you may need to restart the kernel to use updated packages.
/usr/bin/bash: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)
/usr/bin/bash: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)
/usr/bin/bash: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)
chmod: cannot access '/root/.kaggle/kaggle.json': Permission denied
/usr/bin/bash: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/bash)
/usr/bin/bash: line 1: kaggle: command not found


In [ ]:
#unzip
!unzip chess.zip

/usr/bin/zsh: /home/lucas/miniconda3/lib/libtinfo.so.6: no version information available (required by /usr/bin/zsh)
Archive:  chess.zip
  inflating: games.csv               


In [6]:
#format data
text_file = "games.csv"
with open(text_file) as f:
    lines = f.read().split("\n")[:-1]
movesarr = []

for line in lines:
    moves = line.split(",")
    movesarr.append(moves)

In [7]:
#Look at random game
import random

movesarr[random.randint(0, len(movesarr) -1)][12]

'e4 e5 Nf3 Nc6 c3 d5 exd5 Qxd5 Qb3 Nf6 Bc4 Qd7 Ng5 Qd6'

In [8]:
#Filter out too low elo games
moveseq = []

for i in range(len(movesarr) -1): #Minus one because not looking at header in start
    if int(movesarr[i +1][9]) > 1750: #Do plus one because don't look at header
        moveseq.append(movesarr[i][12])
        
len(moveseq)

5746

In [9]:
#Make function to retrieve game sequence array easily
def getgame(num, seq):
    if not seq:
        return moveseq[num]
    else:
        return moveseq[num].split(" ")

In [10]:
def listToString(s):

    # initialize an empty string
    str1 = ""

    # traverse in the string
    for ele in s:
        if ele == s[-1]:
            str1 += f"{ele}"
        else:
            str1 += f"{ele} "
 
    # return string
    return str1

In [11]:
segmented = []
for p in range(5):
    for i in range(len(getgame(p, True))):
        if i +1 != len(getgame(p, True)):
            segmented.append((f"{listToString(getgame(0, True)[:i +1])}", f"[start] {listToString(getgame(0, True)[i+1:i +2])} [end]"))

In [12]:
segmented
#Skal ændres fra at være array bestående af arrays til bare tekst og tekst

[('d4', '[start] d5 [end]'),
 ('d4 d5', '[start] Nc3 [end]'),
 ('d4 d5 Nc3', '[start] Nf6 [end]'),
 ('d4 d5 Nc3 Nf6', '[start] Bf4 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4', '[start] Bf5 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5', '[start] Nf3 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3', '[start] e6 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6', '[start] e3 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3', '[start] Be7 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7', '[start] Bb5+ [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+', '[start] c6 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6', '[start] Bd3 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3', '[start] Bxd3 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3',
  '[start] Qxd3 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3',
  '[start] O-O [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3 O-O',
  '[start] Bg5 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3 

In [13]:
#Partition data
import random
random.shuffle(segmented)

num_val_samples = int(0.15 * len(segmented))
num_train_samples = len(segmented) - 2*num_val_samples
train_samp = segmented[:num_train_samples]
val_samp = segmented[num_train_samples:num_train_samples + num_val_samples]
test_samp = segmented[num_train_samples + num_val_samples:]

In [14]:
#train_start_texts = [pair[0] for pair in train_samp]
#train_end_texts = [pair[1] for pair in train_samp]

In [ ]:
#print(train_start_texts[2])
#print(train_end_texts[2])

#begge længere nede


['e4', 'e5', 'Nf3']
['d5']


In [ ]:
## of samples for each category
print(num_val_samples)
print(num_train_samples)
print(len(test_samp))

25
123
25


In [ ]:
#Make dataset for vocabulary
notationall = []

for i in range(len(moveseq)):
    for p in range(len(getgame(i, True))):
        if getgame(i, True)[p] not in notationall:
            notationall.append(getgame(i, True)[p])

In [ ]:
len(notationall)

3084

In [ ]:
class Vectorizer:
    def tokenize(self, text):
        return text.split()

    def make_vocabulary(self, dataset):
        self.vocabulary = {"": 0, "[UNK]": 1}
        for text in dataset:
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v, k) for k, v in self.vocabulary.items())

    def encode(self, text):
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token, 1) for token in tokens]

    def decode(self, int_sequence):
        return " ".join(
        self.inverse_vocabulary.get(i, "[UNK]") for i in int_sequence)
vectorizer = Vectorizer()
dataset = notationall

vectorizer.make_vocabulary(dataset)

NameError: name 'notationall' is not defined

In [ ]:
#Test it works because fun
test_sentence = train_samp[100]
encoded_sentence = vectorizer.encode(test_sentence)
print(encoded_sentence)
print(vectorizer.decode(vectorizer.encode(test_sentence)))

AttributeError: 'tuple' object has no attribute 'split'

In [ ]:
getgame(0, True)

['e4',
 'e5',
 'Nf3',
 'd5',
 'Nxe5',
 'Nf6',
 'exd5',
 'Bd6',
 'd4',
 'Nxd5',
 'Bc4',
 'Bxe5',
 'dxe5',
 'c6',
 'O-O',
 'Be6',
 'f4',
 'Qb6+',
 'Kh1',
 'Ne3',
 'Bxe3',
 'Qxe3',
 'Bxe6',
 'fxe6',
 'Qd6',
 'Nd7',
 'Qxe6+',
 'Kd8',
 'Rd1']

In [15]:
train_samp

[('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6', '[start] e3 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3 O-O Bg5 Ng4 Bxe7 Qxe7',
  '[start] O-O [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3 O-O Bg5 Ng4 Bxe7 Qxe7 O-O Qg5 h3 Qf6 hxg4 Qh6 Na4 b5 Nc5 Kh8 Nb7 g6 Nd6 Na6 Ne5 Kg8 Ndxf7 Rxf7 Nxf7 Kxf7 f4 g5 fxg5+ Kg7 gxh6+ Kxh6 Rf6+ Kg7 g5 h6 Qg6+ Kh8 Qxh6+ Kg8 Rxe6 Kf7 Qf6+ Kg8 Re8+ Rxe8 Qg6+ Kf8',
  '[start] Qh6+ [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3 O-O Bg5 Ng4 Bxe7 Qxe7 O-O Qg5 h3 Qf6 hxg4 Qh6 Na4 b5',
  '[start] Nc5 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7', '[start] Bb5+ [end]'),
 ('d4 d5 Nc3 Nf6 Bf4', '[start] Bf5 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6 Bd3 Bxd3 Qxd3 O-O Bg5 Ng4 Bxe7 Qxe7 O-O Qg5 h3 Qf6 hxg4 Qh6 Na4 b5 Nc5 Kh8 Nb7 g6 Nd6 Na6 Ne5 Kg8 Ndxf7 Rxf7 Nxf7 Kxf7 f4 g5',
  '[start] fxg5+ [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 Nf3 e6 e3 Be7 Bb5+ c6', '[start] Bd3 [end]'),
 ('d4 d5 Nc3 Nf6 Bf4 Bf5 

In [16]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np

def custom_standardization(input_string):
    return tf.strings.lower(input_string)


vocab_size = 15000
sequence_length = 20


source_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length,
)
target_vectorization = layers.TextVectorization(
    max_tokens=vocab_size,
    output_mode="int",
    output_sequence_length=sequence_length + 1,
    standardize = custom_standardization
)
train_start_texts = [pair[0] for pair in train_samp]
train_end_texts = [pair[1] for pair in train_samp]
source_vectorization.adapt(train_start_texts)
target_vectorization.adapt(train_end_texts)

2022-12-02 23:24:17.070367: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-02 23:24:17.845670: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 2838 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1050 Ti, pci bus id: 0000:01:00.0, compute capability: 6.1


In [17]:
batch_size = 64

def format_dataset(eng, spa):
    eng = source_vectorization(eng)
    spa = target_vectorization(spa)
    return ({
        "english": eng,
        "spanish": spa[:, :-1],
    }, spa[:, 1:])

def make_dataset(pairs):
    eng_texts, spa_texts = zip(*pairs)
    eng_texts = list(eng_texts)
    spa_texts = list(spa_texts)
    dataset = tf.data.Dataset.from_tensor_slices((eng_texts, spa_texts))
    dataset = dataset.batch(batch_size)
    dataset = dataset.map(format_dataset, num_parallel_calls=4)
    return dataset.shuffle(2048).prefetch(16).cache()

train_ds = make_dataset(train_samp)
val_ds = make_dataset(val_samp)

In [18]:
for inputs, targets in train_ds.take(1):
    print(f"inputs['english'].shape: {inputs['english'].shape}")
    print(f"inputs['spanish'].shape: {inputs['spanish'].shape}")
    print(f"targets.shape: {targets.shape}")

inputs['english'].shape: (64, 20)
inputs['spanish'].shape: (64, 20)
targets.shape: (64, 20)


2022-12-02 23:24:21.267207: W tensorflow/core/kernels/data/cache_dataset_ops.cc:856] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


In [19]:
from tensorflow import keras
from tensorflow.keras import layers

embed_dim = 256
latent_dim = 1024

source = keras.Input(shape=(None,), dtype="int64", name="english")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(source)
encoded_source = layers.Bidirectional(
    layers.GRU(latent_dim), merge_mode="sum")(x)

In [20]:
past_target = keras.Input(shape=(None,), dtype="int64", name="spanish")
x = layers.Embedding(vocab_size, embed_dim, mask_zero=True)(past_target)
decoder_gru = layers.GRU(latent_dim, return_sequences=True)
x = decoder_gru(x, initial_state=encoded_source)
x = layers.Dropout(0.5)(x)
target_next_step = layers.Dense(vocab_size, activation="softmax")(x)
seq2seq_rnn = keras.Model([source, past_target], target_next_step)

In [21]:
callbacks = [
    keras.callbacks.ModelCheckpoint("one.keras",
                                    save_best_only=True)
]

seq2seq_rnn.compile(
    optimizer="rmsprop",
    loss="sparse_categorical_crossentropy",
    metrics=["accuracy"],)
seq2seq_rnn.fit(train_ds, epochs=2, validation_data=val_ds, callbacks=callbacks)

Epoch 1/2


2022-12-02 23:24:44.429567: W tensorflow/core/common_runtime/type_inference.cc:339] Type inference failed. This indicates an invalid graph that escaped type checking. Error message: INVALID_ARGUMENT: expected compatible input types, but input 1:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_INT32
    }
  }
}
 is neither a subtype nor a supertype of the combined inputs preceding it:
type_id: TFT_OPTIONAL
args {
  type_id: TFT_PRODUCT
  args {
    type_id: TFT_TENSOR
    args {
      type_id: TFT_FLOAT
    }
  }
}

	while inferring type of node 'cond_41/output/_22'
2022-12-02 23:24:45.261468: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2022-12-02 23:24:46.183336: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f1e194aa070 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2022-12-02 23:24:46.183384: I tensorflow/compiler/

InternalError: Graph execution error:

Detected at node 'StatefulPartitionedCall_12' defined at (most recent call last):
    File "/home/lucas/miniconda3/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/lucas/miniconda3/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/lucas/miniconda3/lib/python3.9/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/lucas/miniconda3/lib/python3.9/site-packages/traitlets/config/application.py", line 985, in launch_instance
      app.start()
    File "/home/lucas/miniconda3/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/lucas/miniconda3/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in start
      self.asyncio_loop.run_forever()
    File "/home/lucas/miniconda3/lib/python3.9/asyncio/base_events.py

In [ ]:
input_sentence = test_eng_texts[1]
#print("-")
#print(input_sentence)
#print(decode_sequence(input_sentence))

tokenized_input_sentence = source_vectorization([input_sentence])
decoded_sentence = "[start]"
tokenized_target_sentence = target_vectorization([decoded_sentence])
next_token_predictions = seq2seq_rnn.predict(
    [tokenized_input_sentence, tokenized_target_sentence], verbose=None)
sampled_token_index = np.argmax(next_token_predictions[0, 20, :])
sampled_token = spa_index_lookup[sampled_token_index]

print(sampled_token)

[end]
